In [1]:
import sys
sys.path.append('../input/strip-ai-seam-carving/seam-carving')

In [2]:
import os, random, gc
import seaborn as sns
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn # neural network module
import torch.nn.functional as F # neural network module에서 자주 사용되는 함수
import torchvision
import cv2, math, shutil # OpenCV => cv2
import albumentations as Albu
import rasterio
import seam_carving
import skimage
from skimage.io.manage_plugins import call_plugin
from skimage.color import rgb2hed, hed2rgb
from skimage.exposure import rescale_intensity

import matplotlib.pyplot as plt
from rasterio.enums import Resampling
from rasterio.transform import Affine
from torchvision import models
from torchvision import transforms
from albumentations.pytorch import ToTensorV2  
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score
from tqdm.notebook import tqdm
%matplotlib inline

In [3]:
# Step 1.1 Test, Submission Path Setting
data_path = '../input/mayo-clinic-strip-ai/'
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')
submission_efficient = pd.read_csv(data_path + 'sample_submission.csv')
submission_vit = pd.read_csv(data_path + 'sample_submission.csv')

In [4]:
# Step 1.1.1 Seed Fasten
seed = 50
os.environ['PYTHONHASHSEED'] = str(seed) # python 난수 고정 => 환경변수 접근, 고정 시드 넘버 할당
random.seed(seed) # random module 난수 고정
np.random.seed(seed) # numpy module 난수 고정
torch.manual_seed(seed)

torch.manual_seed(seed) # 파이토치 CPU 난수 생성기 => 시드 고정 
torch.cuda.manual_seed(seed) # 파이토치 GPU 난수 생성기 => 시드 고정
torch.cuda.manual_seed_all(seed) # 파이토치 멀티 코어_GPU 난수 생성기 => 시드 고정

torch.backends.cudnn.deterministic = True # 확정적 연산 사용
torch.backends.cudnn.benchmark = False # benchmark function 해제
torch.backends.cudnn.enabled = False # cudnn 사용 해제

In [5]:
# Step 1.2.1 Device setting => GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
dir_path = './test'
os.mkdir(dir_path)

In [7]:
# Demo Test Data Set Convert (Tiff to Png)
image_scalar = 1024
seam_carving.carve.MAX_MEAN_ENERGY = 10.0
# fig, axes = plt.subplots(2, 2, figsize=(7, 6), sharex=True, sharey=True)
# ax = axes.ravel()
for i in tqdm(range(test.shape[0])):
    # Remove Background White Space
        img_id = test.iloc[i].image_id
        img_path = f'../input/mayo-clinic-strip-ai/test/{img_id}.tif'

        image = rasterio.open(img_path)
        image = image.read(out_shape=(image.count, int(image_scalar), int(image_scalar)),
                                 resampling=Resampling.bilinear).transpose(1,2,0)
        image_h, image_w, _ = image.shape
        image = seam_carving.resize(image, (image_w-512, image_h-512),
                                        energy_mode='backward',
                                        order=('width-first'),
                                        keep_mask=None)
    # Apply Stain Color Normalization => Using Skimage
        ihc_hed = rgb2hed(image)
        null = np.zeros_like(ihc_hed[:, :, 0])
        h = rescale_intensity(ihc_hed[:, :, 0], out_range=(0, 1),
                      in_range=(0, np.percentile(ihc_hed[:, :, 0], 99)))
        d = rescale_intensity(ihc_hed[:, :, 2], out_range=(0, 1),
                      in_range=(0, np.percentile(ihc_hed[:, :, 2], 99)))
        # Cast the two channels into an RGB image, as the blue and green channels
        # respectively
        image = np.dstack((null, d, h))#.transpose(2,0,1).astype(np.uint16)
        print(image.shape)
#         fig = plt.figure()
#         axis = plt.subplot(1, 1, 1, sharex=ax[0], sharey=ax[0])
#         axis.imshow(image)
#         axis.set_title('Stain-separated image (rescaled)')
#         axis.axis('off')
#         plt.show()
#         with rasterio.open(f'./test/{img_id}.png', 'w', driver='png', height = image.shape[1], # driver => jpeg, png, gtiff
#                        width = image.shape[2], dtype = image.dtype, count=3) as images: # count => Image Channel 개수 (RGB의 경우 3개)
#             images.write(image)

        skimage.io.imsave(f'./test/{img_id}.png', image)
        del image
        gc.collect()
        
        

#         if img_id == '006388_0':
#             target_image = image
#             image = image.transpose(2,0,1)
#             with rasterio.open(f'./test/{img_id}.png', 'w', driver='png', height = image.shape[1], # driver => jpeg, png, gtiff
#                            width = image.shape[2], dtype = image.dtype, count=3) as images: # count => Image Channel 개수 (RGB의 경우 3개)
#                 images.write(image)

#             continue
        
#         else:
#             target_img = staintools.LuminosityStandardizer.standardize(target_image)
#             transform_img = staintools.LuminosityStandardizer.standardize(image)

#             normalizer = staintools.StainNormalizer(method='vahadane')
#             target_img_n = normalizer.fit(target_img)
#             transform_img_n = normalizer.transform(transform_img)
#             image = transform_img_n.transpose(2,0,1)
#             with rasterio.open(f'./test/{img_id}.png', 'w', driver='png', height = image.shape[1], # driver => jpeg, png, gtiff
#                        width = image.shape[2], dtype = image.dtype, count=3) as images: # count => Image Channel 개수 (RGB의 경우 3개)
#                 images.write(image)

#             del image
#             gc.collect()

  0%|          | 0/4 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)


In [8]:
# Step 1.2.2 Definition Data Set Class
class ImageDataset(Dataset):
    def __init__(self, df, img_dir='./', transform=None, is_test=True):
        super().__init__()
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
    
        if self.is_test:
            img_id = self.df.iloc[idx, 0] 
            img_path = f'{self.img_dir}/{img_id}.png'
            
            image = rasterio.open(img_path)
            image = image.read(resampling=Resampling.bilinear).transpose(1,2,0)
            patient_ids = self.df.iloc[idx, 2]
            
            if self.transform is not None:
              image = self.transform(image=image)['image']
            
            return image, patient_ids 

In [9]:
# Step 1.3 Test Transform
transform_test = Albu.Compose([Albu.Normalize(mean=[0.485, 0.456, 0.406], 
                                              std=[0.229, 0.224, 0.225]),
                               ToTensorV2()])

In [10]:
# 1.4 Test Data Set
img_dir = './test'
dataset_test = ImageDataset(test, img_dir=img_dir, transform=transform_test)

In [11]:
# Step 1.5 Data Loader Definition 
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() %2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

batch_size = 1
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False, 
                         worker_init_fn=seed_worker, generator=g, num_workers=1)

In [12]:
# Step 1.6 Pretrained Model => Efficient Net-B5
# 앙상블 추후 적용 => 모델 리스트화
model = []

# Efficient-Net
efficient_net = models.efficientnet_b0()
efficient_net.classifier = nn.Linear(1280,2)
efficient_net.load_state_dict(torch.load('../input/strip-ai-train-parameters/new_efficient_model_state_dict.pth'))
print(efficient_net.classifier)

efficient_net = efficient_net.to(device)
model.append(efficient_net)

Linear(in_features=1280, out_features=2, bias=True)


In [13]:
# regnet_x = models.regnet_x_8gf()
# regnet_x.fc = nn.Linear(1920,2)
# regnet_x.load_state_dict(torch.load('../input/strip-ai-regnet-train-1k-png/model_state_dict.pth'))
# print(regnet_x.fc)

# regnet_x = regnet_x.to(device)
# model.append(regnet_x)

In [14]:
vit = models.vit_l_16(image_size=512)
vit.heads = nn.Linear(1024,2)
vit.load_state_dict(torch.load('../input/strip-ai-vit-train-parameter/vit_model_state_dict.pth'))

vit = vit.to(device)
model.append(vit)
# i want to test Vision Transformer model but, computing powers that i have are limited.... 

In [15]:
# Step 1.7 Predict Function Definition
def predict(model, loader_test): 
    ids = []
    model.eval()
    preds_part = []
    
    with torch.no_grad():
        for images, patient_ids in loader_test:
            images = images.to(device) #, dtype=torch.float32)
            outputs = model(images)
            
            ids.append(patient_ids[0]) # Pytorch Broadcasting => patient_ids 차원 추가, 그래서 ('patient_id',) <= 출력 형태가 저런 모습이었다. 
            preds_part.append(torch.softmax(outputs.cpu(), dim=1).squeeze().numpy())
            
    return np.array(preds_part), ids

In [16]:
# Step 3.8.6 Submission
preds, ids = predict(model[0], loader_test)
result_table_efficient = pd.DataFrame({'patient_id' : ids, "CE" : preds[:,0], "LAA" : preds[:,1]})
result_table_efficient = result_table_efficient.groupby("patient_id").mean()

In [17]:
result_table_efficient

,CE,LAA
patient_id,,
006388,0.817273,0.182727
008e5c,0.885027,0.114973
00c058,0.485383,0.514617
01adc5,0.736614,0.263386


In [18]:
submission_efficient.CE = result_table_efficient.CE.to_list()
submission_efficient.LAA = result_table_efficient.LAA.to_list()

#submission_efficient[['patient_id', 'CE', 'LAA']].round(6).to_csv('submission.csv', index=False)
submission_efficient

,patient_id,CE,LAA
0,006388,0.817273,0.182727
1,008e5c,0.885027,0.114973
2,00c058,0.485383,0.514617
3,01adc5,0.736614,0.263386


In [19]:
preds, ids = predict(model[1], loader_test)
result_table_vit = pd.DataFrame({'patient_id' : ids, 'CE' : preds[:,0], 'LAA' : preds[:,1]})
result_table_vit = result_table_vit.groupby("patient_id").mean()

/opt/conda/lib/python3.7/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [20]:
result_table_vit

,CE,LAA
patient_id,,
006388,0.267623,0.732377
008e5c,0.270028,0.729972
00c058,0.285510,0.714490
01adc5,0.294902,0.705098


In [21]:
submission_vit.CE = result_table_vit.CE.to_list()
submission_vit.LAA = result_table_vit.LAA.to_list()
submission_vit

,patient_id,CE,LAA
0,006388,0.267623,0.732377
1,008e5c,0.270028,0.729972
2,00c058,0.285510,0.714490
3,01adc5,0.294902,0.705098


In [22]:
submission.CE = (submission_efficient.CE + submission_vit.CE) / 2
submission.LAA = (submission_efficient.LAA + submission_vit.LAA) / 2

submission[['patient_id', 'CE', 'LAA']].round(6).to_csv('submission.csv', index=False) # CE, LAA만 괄호에 넣었더니 patient_id이 사라진 상태로 CSV 파일이 저장된 경우....
submission

,patient_id,CE,LAA
0,006388,0.542448,0.457552
1,008e5c,0.577528,0.422472
2,00c058,0.385446,0.614554
3,01adc5,0.515758,0.484242


In [23]:
!head submission.csv

patient_id,CE,LAA
006388,0.542448,0.457552
008e5c,0.577528,0.422472
00c058,0.385446,0.614554
01adc5,0.515758,0.484242
